In [2]:
import numpy as np
import pygal
import torch.nn as nn
import torch
import matplotlib.pyplot as plt

In [58]:
n = 500
x = np.linspace(-6,6,n)
y = np.cos(2*x) + np.sin(4*x)
# x = (x-x.mean())/x.std()
x_,y_ = torch.Tensor(x.reshape(-1,1)).to(torch.float32).cuda(), torch.Tensor(y.reshape(-1,1)).to(torch.float32).cuda()

In [59]:
np.random.choice(np.arange(10),5,replace=True)

array([9, 4, 7, 9, 7])

In [60]:
class net(nn.Module):
    
    def __init__(self):

        super(net,self).__init__()
        self.l1 = nn.Linear(1,16)
        self.l2 = nn.Sigmoid()
        self.l3 = nn.Linear(16,4)
        self.l4 = nn.Sigmoid()
        self.l5 = nn.Linear(4,1)
        # self.l6 = nn.Sigmoid()

    def forward(self,X):

        x=X

        x = self.l1(x)
        x = self.l2(x)
        x = self.l3(x)
        x = self.l4(x)
        x = self.l5(x)
        # x = self.l6(x)

        
        return x

In [76]:
model = net()
learning_rate = 0.1

model = model.to(torch.float32)

loss = nn.MSELoss()
optim = torch.optim.Adam(model.parameters(), lr=learning_rate)

def iterate():

    global x,y
    x_ = torch.tensor(x).to(torch.float32).reshape(-1,1)
    y_ = torch.tensor(y).to(torch.float32).reshape(-1,1)

    output = model(x_)

    # print(output)
    j = loss(output, y_)

    optim.zero_grad()
    j.backward()
    optim.step()

    return output.detach().numpy()


def animate(frames):
    
    output = iterate()

    if frames%50==0:
        plt.clf()
        global x,y
        plt.plot(x,y)
        plt.plot(x,output)
        plt.title(f"Iteration {frames}")

In [77]:
from matplotlib.animation import FuncAnimation

%matplotlib qt
fig1 = plt.figure()
ani = FuncAnimation(fig1,animate,frames=500,interval=1)
ani.save("../static/hm.gif")

MovieWriter ffmpeg unavailable; using Pillow instead.


In [37]:
%matplotlib inline

In [43]:
import io
import matplotlib
matplotlib.use("Agg")


for i in range(500):
    output = iterate()
    plt.plot(x,output)
    buf = io.BytesIO()
    plt.savefig(buf,dpi=72)
    

KeyboardInterrupt: 

In [34]:
h = fig.canvas.tostring_rgb()

In [18]:
yhat = np.array(model(x_).detach().numpy())

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [ ]:
l = [(x[i],y[i]) for i in range(n)]
l1 = [(x[i],yhat[i].item()) for i in range(n)]
chart = pygal.XY(width=2000,height=500,stroke=False)
chart.add("Ground Truth",l)
chart.add("Predicted",l1)
chart.render_in_browser()